<h1><center>CTA Forecasting</center></h1>
<h2><center>Long Term Project</center></h2>

# Objective

In this project you will create a kitchen sink classification and regression models that forecast the optimal trading positions for Oil Futures. Your framework should be generic enough so as to be testable on other markets. You will try the same methods for Sugar, Copper, Gold, Natural Gas.

# Data
The datasets are in the datasets/futures. Make sure you talk to the Futures data gathering team in order to ensure that you are processing the data properly.


# Motivation
For every Futures asset, the contracts that are being traded at time $t$ are indexed by the Futures expiration $T$, which in this case you will take to be the last available trading date of each contract. Because each Futures expires at $T$, the returns of each individual Futures contract is not time-homogeneous. In fact since we have:

$$
E_t[S_T] = F^T_t \qquad (1)
$$

then as $t\rightarrow{T}$, the Futures price should converge to the spot price. This of course assumes zero convenience yield costs such as storage and cargo costs, etc. In general, however, different futures have different storage costs. Gold, for example, is the easiest to store and transport, so according to the above equation the Gold futures price should converge fairly accurately to the actual price of an ounce of gold. Gas is one of the most difficult to store, so the spot isn't quite easily tradable. Moreover, the storage costs of Gas exhibit large seasonality effects (e.g. see the [EIA Weekly Natural Gas Storage Report](http://ir.eia.gov/ngs/ngs.html)), which also translates into seasonality in the Futures curve.

Let's consider a non-seasonal Futures such as Oil. Daily fluctuations of demand for oil translate into large price shocks which are bigger for expirations close to $T$. and such behavior is common for all commodity Futures. Mathematically:

$$
dF^T_t = \mu_t(t,T)dt + \sigma_t(t,T)dW^T_t \qquad(2)
$$
If there is no seasonality, both $\mu(t,T)$ and $\sigma_t(t,T)$ would be time-homogeneous, i.e. $\mu_t(t,T)=\mu_t(T-t)\equiv\mu_t(\tau)$ and $\sigma_t(t,T)=\sigma_t(T-t)\equiv\sigma_t(\tau)$, where $\tau\equiv{T-t}$. Bigger shocks at short expirations means that for fixed $t$, $\sigma$ is decreasing as a function of the $relative$ $expiration$ $\tau$. The above model is fairly general as $\mu_t$ and $\sigma_t$ are in generall allowed to be any time-homogeneous stochastic functions!

In terms of relative expiration coordinates, we can express the Futures dynamics of $F_t(\tau)\equiv{F}^{t+\tau}_t$ as (remember Ito):

$$
\begin{align}
dF_t(\tau) &= d_tF_t(\tau) + d_\tau{F}_t(\tau) + \frac{1}{2}\partial^2_{t,\tau}F_t(\tau){\rm Cov}E[dt,d\tau]\\
&=-\partial_\tau{F}dt+dF^T_t \qquad (3)
\end{align}
$$
where the second term in the last line above is given by $(2)$. Make sure you understand why the second line above follows from the first one (Hint: what is $E[dtd\tau]$?). The first term in the last line of (3) is called $carry$. It can be interpreted as the cost of rolling our Futures one day forward so as to maintain the same $relative$ $expiration$ $\tau$. The magnitude of this carry is known at $t$ from interpolating the Futures curve and taking its first derivative. The second term, on the other hand, is not known in advance at $t$. This is the stochastic shock of the curve.

The goal of this project for you is two fold. 
- Your first goal is to find models that help you estimate the alpha $\alpha_\tau\equiv\mu(T-t)$ for every relative expiration contract at $\tau$. You will in general construct relative expiration contracts at every month, e.g. $\tau_i=30, 60, 90,..$ days. To do that you will have to do daily roll of the Futures contracts corresponding to linear interpolation of the curve.
- However knowing $\alpha_i\equiv\alpha_{\tau_i}$ is not enough to find the optimal trading strategy for a given Futures curve. You will also have to build a covariance model, e.g. estimate, $\Sigma_{i,j}=Cov[dF_t(\tau_i)dF_t(\tau_i)]$ and build a risk model. Only after you build your risk model you can find the exposures that produce the optimal Sharpe:
$$
\omega^*_i(t) \propto \sum_j\Sigma^{-1}_{ij}\alpha_j, \quad \sum_i\omega^*_i=1
$$
and plot the PnL of the strategy $d\Pi_t\equiv\sum_i\omega^*_i(t)dF_t(\tau_i)$.



# Methodology and Deliverables
## Roll the Futures
In this part you will have to create a script which rolls your Futures contracts and obtains the Futures timeseries $dF_t(\tau_i)$ for each $\tau_i$. Your output will be a data frame table of df_rolled indexed by [date] with columns [F_1, F_2,...,F_N] where N is the maximum number of futures contracts so that the Ndays x N table F has less than 0.1% nans.

The time series of the $n$-th rolled contract $F_n$, at time $t+1$ corresponds to the PnL of the following dynamic strategy:
$$
dF_n(t) = \frac{\Delta{t}}{T_{n+1}-T_n}dF^{T_n}_t + (1-\frac{\Delta{t}}{T_{n+1}-T_n})dF^{T_{n+1}}_t,\quad{\Delta{t}}\equiv{T}_1-t
$$
Note that in the above formula $\Delta{t}=T_1 - t$ is the time to maturity of the $front$ contract. Implicitly, in the above we are assuming that $T_1-t\leq{T}_{n+1}-T_n$ for all n. So double check this is always the case before you roll your contracts, as some contracts may be missing from the chain.

If any of $dF^{T_n}_t$ or $dF^{T_{n+1}}_t$ are nan's, then $dF_n(t)$ is a nan. At the end, since no time series $dF_n(t)$ has more than 1% of nans, we can assume zero return on those dates (which would produce a small underestimation of risk and perhaps a tiny error in alpha). Generally this procedure corresponds to marking your illiquid side of the book as constant (often a common practice even for large institutions). 

Plot the time series of the front 12 months worth of contracts for the various Futures classes you are looking at. Are any of the contracts exibiting drifts? Do any of them trade within range?

## Estimate a Covariance model
In this section, for each asset class in question, estimate a covariance model over a 2 year rolling window, rolled every month. In other words, compute the spectrum of your covariance matrix for each rolling window. How much variance do the top 5 factors capture for each of your asset classes? Do you see any peculiarities for your asset classes? 

- Store the entire spectrum for each rolling window in a dataframe df_evals_assetname indexed by end_date (the last date of the rolling window) and having columns ev_1,...., ev_N.
- Store the top 5 eigenvectors of the spectrum for each rolling window in a dataframe df_evecs_assetname indexed by [end_date, evnum] and containing the weights of each eigenvector w_1 ,...., w_N as columns. Note that $\sum^N_{n=1} w^2_n=1$

## Filter Down Your Signals and Tradables Using SVD
### What You Will Do
- Read the SVD Application Notes that we went over in class.
- Look at various alpha factors:
    - the futures curve at time t
    - yesterday's return of all your rolled futures
    - the 5-day return of your rolled futures
    - the 20-day return of your rolled futures
    - the 30-day annualized volatility of your rolled futures
- Combine the following return variables to be forecasted:
    - today's return of all your rolled futures
    - next 5 days' return of of all your rolled futures
    - next 20 days' return of all your rolled futures
    - today's return of the top 3 covariance factors, with covariance estimated up to yesterday as in the previous section
    - next 5 days' return of the top 3 covariance factors, with covariance estimated up to yesterday as in the previous section
    - next 20 days' return of the top 3 covariance factors, with covariance estimated up to yesterday as in the previous section
- Do the following study:
    - On a 1-year rolling window where you roll every month, compute ${\bf G}$ and its SVD as in the notes. Store the spectrum in a df_evals_G matrix as you did in the previous section. Which eigenvalues are signifiant?
    - Store the top 5 eigenectors in a dataframe df_evecs_G as before.
    - Which forecasting variables produce the biggest explanatory power of which returns?  
    
## Build an Alpha Model
- After you understand which forecasting variables are the most important ones, build an alpha regression model (Equation (1) in the SVD notes) with these variables. Make sure you properly regularize within an Elastic Net model as to ensure you don't get singular matrix because of factor colinearity.
    - Find the optimal set of parameters for your alpha model over the training set
    - Plot the in-sample PnL for the optimal model over the training set
    - What is the PnL of your model over the out of sample test set?
- Also, build a multi-class classification model to predict which of your tradables will have the highest return and which of yout taradables will have the lowest. 
    - In other words, your traning classification variable $y$ is $+1$ for those assets which are in the top 10% of performers, $-1$ for those assets in the bottom 10% of performers and $0$ for the rest.
    - Train SGDClassifier with hinge and log loss. Compute the confusion matrix for each model. Vary whatever model parameters are tunable in your model in order to produce an optimal model.
    - For the optimal model within the training sample, what is the PnL of the strategy that goes long the top 10% and short the bottom 10% predicted assets? How does the PnL compare to the regression strategy above.
    - Finally, test the performance of the optimal model on the test set.
- Which approach is more appropriate for your datasets? The regression approach or the classification approach?

### README file and code

Create a README_db.txt file containing:
- The names and emails of all the teammates so you can be contacted by the next user of the dataset
- Description of your code base and how it can be used by others
- Outline who was resposible for which part of the project

Make sure you document your notebooks with explanations so others in the class can use them!
